# Lesson 3: Reflection and Blogpost Writing

## Setup

In [1]:
llm_config = {"model": "gpt-3.5-turbo"}

## The task!

In [2]:
task = '''
        Write a concise but engaging blogpost about
       Microsoft. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [3]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [4]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [5]:
print(reply)

Title: "Exploring the Evolution of Microsoft"

Discover the revolutionary journey of Microsoft in just a few decades. From software giant to tech powerhouse, Microsoft has transformed the way we work, connect, and create. With innovations like Windows, Xbox, and Office, it continues to shape our digital future. Under CEO Satya Nadella's leadership, the company has embraced AI, cloud computing, and more. Microsoft's commitment to accessibility and sustainability sets it apart in the tech industry. As we witness its ongoing growth and impact, one thing is clear: Microsoft is not just a company; it's a driving force of innovation in the modern world.


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [6]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [7]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       Microsoft. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Exploring the Evolution of Microsoft"

Discover the revolutionary journey of Microsoft in just a few decades. From software giant to tech powerhouse, Microsoft has transformed the way we work, connect, and create. With innovations like Windows, Xbox, and Office, it continues to shape our digital future. Under CEO Satya Nadella's leadership, the company has embraced AI, cloud computing, and more. Microsoft's commitment to accessibility and sustainability sets it apart in the tech industry. As we witness its ongoing growth and impact, one thing is clear: Microsoft is not just a company; it's a driving force of innovation in the modern world.

--------------------------------------------------------------------------------
Critic (

## Nested chat

In [9]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [10]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [12]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [13]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [14]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [15]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [16]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       Microsoft. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Exploring the Evolution of Microsoft"

Discover the revolutionary journey of Microsoft in just a few decades. From software giant to tech powerhouse, Microsoft has transformed the way we work, connect, and create. With innovations like Windows, Xbox, and Office, it continues to shape our digital future. Under CEO Satya Nadella's leadership, the company has embraced AI, cloud computing, and more. Microsoft's commitment to accessibility and sustainability sets it apart in the tech industry. As we witness its ongoing growth and impact, one thing is clear: Microsoft is not just a company; it's a driving force of innovation in the modern world.

--------------------------------------------------------------------------------

*******


--------------------------------------------------------------------------------
Meta Reviewer (to Critic):

Aggregrating feedback from all reviewers, it is clear that the writing can benefit from including targeted keywords related to Microsoft, optimizing meta tags, and adding internal links for better SEO value and user engagement. Additionally, ensuring that all information provided is accurate and supported by credible sources, avoiding biased language, and maintaining transparency regarding any potential conflicts of interest or affiliations related to Microsoft are important suggestions.

Considering these feedback, it is recommended to incorporate targeted keywords, optimize meta tags, and add internal links for improved search visibility and user engagement. It is crucial to ensure the accuracy of information, avoid biased language, and maintain transparency to enhance credibility and trustworthiness in the content. By addressing these aspects, the writing can be more effecti

## Get the summary

In [17]:
print(res.summary)

Title: "Unveiling the Transformation of Microsoft: A Tech Evolution Story"

Embark on a riveting journey through Microsoft’s remarkable evolution over the years. From pioneering software solutions to becoming a global tech powerhouse, Microsoft's impact knows no bounds. With groundbreaking innovations such as Windows, Xbox, and Office, the company continues to redefine the digital landscape. Guided by CEO Satya Nadella, Microsoft delves into cutting-edge technologies like AI and cloud computing, shaping a more connected future. Championing accessibility and sustainability, Microsoft stands at the forefront of responsible tech leadership. Witness firsthand how Microsoft transcends boundaries, not just as a corporation, but as a beacon of innovation in the digital age.
